In [1]:
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import Column, Integer, String, Float, Time
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
# from config import username, password
from flask import Flask, jsonify, render_template


In [2]:
csv_file = "data/2019_Ironman_World_Championship_Results.csv"
race_stats_df = pd.read_csv(csv_file)
csv_file = "data/countries_codes_and_coordinates.csv"
countries_df = pd.read_csv(csv_file)

In [3]:
# merge both dataframes into one dataframe on country code
merged_df = pd.merge(race_stats_df, countries_df, left_on="Country", right_on="Alpha_3_code")

In [13]:
# Remove unnecessary columns and rename columns
ironman_df = merged_df.loc[:, ['BIB', 'Last_Name', 'First_Name', 'Country_x', 'Country_y', 'Gender', 'Division', 'Swim', 'Bike', 'Run', 'Overall', 'Division_Rank', 'Gender_Rank', 'Overall_Rank', 'T1', 'T2', 'Latitude_average', 'Longitude_average']].dropna()
ironman_df = ironman_df.rename(columns={"Country_x":"Alpha_3_code", "Country_y":"Country"})


In [14]:
ironman_df

,BIB,Last_Name,First_Name,Alpha_3_code,Country,Gender,Division,Swim,Bike,Run,Overall,Division_Rank,Gender_Rank,Overall_Rank,T1,T2,Latitude_average,Longitude_average
0,4,Frodeno,Jan,DEU,Germany,Male,MPRO,0:47:31,4:16:02,2:42:43,7:51:13,1,1,1,0:01:58,0:02:59,51.0,9.0000
1,5,Kienle,Sebastian,DEU,Germany,Male,MPRO,0:52:17,4:15:04,2:49:56,8:02:04,3,3,3,0:02:12,0:02:35,51.0,9.0000
2,45,Frommhold,Nils,DEU,Germany,Male,MPRO,0:50:47,4:24:29,3:05:10,8:24:56,20,20,20,0:02:08,0:02:22,51.0,9.0000
3,44,Duelsen,Marc,DEU,Germany,Male,MPRO,0:52:21,4:28:26,3:01:14,8:26:35,23,23,23,0:02:07,0:02:27,51.0,9.0000
4,3003,Haug,Anne,DEU,Germany,Female,FPRO,0:54:09,4:50:17,2:51:07,8:40:10,1,1,32,0:02:02,0:02:35,51.0,9.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,1331,Altayeb,Dina,SAU,Saudi Arabia,Female,F45-49,1:36:44,7:11:32,5:46:14,14:51:02,80,538,2087,0:07:06,0:09:26,25.0,45.0000
2471,402,Fernandez vasquez,Bianca,DOM,Dominican Republic,Female,F35-39,1:58:24,7:29:58,5:17:15,15:00:22,73,545,2101,0:05:59,0:08:46,19.0,-70.6667
2472,327,Vu,Heidi,VNM,Viet Nam,Female,F45-49,1:22:41,7:48:27,6:17:47,15:40:02,84,574,2163,0:05:14,0:05:53,16.0,106.0000
2473,327,Vu,Heidi,VNM,Vietnam,Female,F45-49,1:22:41,7:48:27,6:17:47,15:40:02,84,574,2163,0:05:14,0:05:53,16.0,106.0000


In [15]:
ironman_df.dtypes

BIB                    int64
Last_Name             object
First_Name            object
Alpha_3_code          object
Country               object
Gender                object
Division              object
Swim                  object
Bike                  object
Run                   object
Overall               object
Division_Rank         object
Gender_Rank           object
Overall_Rank          object
T1                    object
T2                    object
Latitude_average     float64
Longitude_average    float64
dtype: object

In [27]:
ironman_df.loc[:, 'Swim'] = pd.to_datetime(ironman_df.loc[:, 'Swim'], format='%H:%M:%S') - pd.to_datetime(ironman_df.loc[:, 'Swim'], format='%H:%M:%S').dt.normalize()
ironman_df.loc[:, 'Bike'] = pd.to_datetime(ironman_df.loc[:, 'Bike'], format='%H:%M:%S') - pd.to_datetime(ironman_df.loc[:, 'Bike'], format='%H:%M:%S').dt.normalize()
ironman_df.loc[:, 'Run'] = pd.to_datetime(ironman_df.loc[:, 'Run'], format='%H:%M:%S') - pd.to_datetime(ironman_df.loc[:, 'Run'], format='%H:%M:%S').dt.normalize()

In [28]:
clean_df.dtypes

BIB                           int64
Last_Name                    object
First_Name                   object
Alpha_3_code                 object
Country                      object
Gender                       object
Division                     object
Swim                 datetime64[ns]
Bike                 datetime64[ns]
Run                  datetime64[ns]
Overall                      object
Division_Rank                object
Gender_Rank                  object
Overall_Rank                 object
T1                           object
T2                           object
Latitude_average            float64
Longitude_average           float64
dtype: object

In [29]:
swim = ironman_df['Swim'].mean()
swim

Timedelta('0 days 01:12:17.557105')

In [30]:
bike = ironman_df['Bike'].mean()
bike

Timedelta('0 days 05:45:38.909328')

In [25]:
run = ironman_df['Run'].min()
run

Timestamp('1900-01-01 02:42:43')

In [7]:
engine = create_engine("sqlite:///data/ironman.sqlite")

In [8]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [9]:
Base.classes.keys()

[]

In [10]:
# Save reference to the table
Race_Stats = Base.classes.race_stats

AttributeError: race_stats

In [12]:
session = Session(engine)